<a href="https://colab.research.google.com/github/amodsgit/AmodTheCoder/blob/main/hair_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For any inquiries regarding this code, please feel free to contact me at amods98@icloud.com

In [2]:
!pip install -qU langchain openai pinecone-client

In [3]:
import openai
PINECONE_API_KEY = "e02414a4-a7de-46da-927d-632b8008c9ab"  # Pinecone API Key
PINECONE_ENV = 'us-west4-gcp-free'

index_name = "haircare-chat-sit"

import os

os.environ['OPENAI_API_KEY'] = '<Replace with your API key>' #Get an OpenAI API key from platform.openai.com

openai.api_key = os.getenv("OPENAI_API_KEY") or "OPENAI_API_KEY"

In [4]:
import pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV,
)
pinecone.whoami()

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


WhoAmIResponse(username='24b6755', user_label='default', projectname='fdd0d9a')

In [5]:
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 273}},
 'total_vector_count': 273}

In [22]:
import openai
import pprint
import json
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
# Embedding model
embed_model = "text-embedding-ada-002"

# Your question
query = "What are the side effects of Mupirocin?"

# Get the query embedding
res = openai.Embedding.create(
    input=[query],
    engine=embed_model
)

# Retrieve the embedding from the response
xq = res['data'][0]['embedding']

# Use Pinecone to find the top 5 relevant contexts
res = index.query(xq, top_k=1, include_metadata=True)

# Extract the retrieved contexts
contexts = [item['metadata'] for item in res['matches']]

# Convert dictionaries to strings
contexts = ["\n".join(f"{key}: {value}" for key, value in context.items()) for context in contexts]

# System primer message
primer = {
    'role': 'system',
    'content': """You are a sophisticated AI, capable of understanding complex medical information and responding to queries based on the context. The information you provide is based strictly on the data available within the conversation. If specific information has not been provided within the conversation, you cannot access it. Be sure to reference details provided within the context when forming your responses and be transparent when you don't have enough information to provide a reliable answer."""
}

# User message
user_message = {
    "role": "user",
    "content": query
}

# Context message
context_messages = [{
    "role": "assistant",
    "content": context
} for context in contexts]

# Messages to be sent to the API
messages = [primer, *context_messages, user_message]

# Print the formatted messages
pprint.pprint(messages)


[{'content': 'You are a sophisticated AI, capable of understanding complex '
             'medical information and responding to queries based on the '
             'context. The information you provide is based strictly on the '
             'data available within the conversation. If specific information '
             'has not been provided within the conversation, you cannot access '
             'it. Be sure to reference details provided within the context '
             "when forming your responses and be transparent when you don't "
             'have enough information to provide a reliable answer.',
  'role': 'system'},
 {'content': 'hair_disease: Folliculitis\n'
             'medication: Mupirocin\n'
             'severity: Mild\n'
             'side_effects: Burning sensation, Itching, Redness\n'
             'symptoms: Red and swollen hair follicles, itchy or burning skin, '
             'pus-filled blisters that break open and crust over.',
  'role': 'assistant'},
 {'conte

In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
primer_str = primer['content']
final_prompt = primer_str + '\n' + 'User query: ' + query + '\n' + 'Context: ' + '{' + ' '.join(contexts) + '}'  #create the final propt of the first message

print(final_prompt)

You are a sophisticated AI, capable of understanding complex medical information and responding to queries based on the context. The information you provide is based strictly on the data available within the conversation. If specific information has not been provided within the conversation, you cannot access it. Be sure to reference details provided within the context when forming your responses and be transparent when you don't have enough information to provide a reliable answer.
User query: What are the side effects of Mupirocin?
Context: {hair_disease: Folliculitis
medication: Mupirocin
severity: Mild
side_effects: Burning sensation, Itching, Redness
symptoms: Red and swollen hair follicles, itchy or burning skin, pus-filled blisters that break open and crust over.}


In [24]:

llm = ChatOpenAI(
    temperature=0,  #randomness of the llm response
    model='gpt-3.5-turbo'
)

conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferWindowMemory(k=1) # we can use
)

res = conversation(final_prompt)

In [25]:
print(res['response'])

The side effects of Mupirocin, based on the context provided, include a burning sensation, itching, and redness. These side effects may occur when using Mupirocin to treat folliculitis, a hair disease characterized by red and swollen hair follicles, itchy or burning skin, and pus-filled blisters that break open and crust over.


In [26]:
res2 = conversation('what is it prescribed for?') # second and subsequent message are passed directly without semantic search

In [27]:
print(res2['response'])

Mupirocin is commonly prescribed to treat bacterial skin infections, such as impetigo and folliculitis. It is an antibiotic ointment that works by killing or stopping the growth of certain bacteria on the skin.


In [28]:
res3 = conversation('what should i do in case of side effects?')

In [29]:
print(res3['response'])

If you experience any side effects while using Mupirocin, it is important to consult your healthcare provider. They will be able to provide guidance on what steps to take. In general, common side effects may include itching, burning, or stinging at the application site. If these side effects persist or worsen, it is important to seek medical attention.
